<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/three_plus_word_resp_2nd_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove posts with only 1 or 2 words. Leave response length at 20.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "three_plus_words"
MODEL_NAME = "three_plus_words"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
import random
import math
import os
!pip install gcsfs

pd.set_option('max_colwidth', 100)



In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
test_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/test.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
test_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
0,1682766,78379937,W,21367,Tonia,jacky thank you for addressing this very troubling news! please demand a full independent invest...,Jacky Rosen,Congress_Democratic
1,1682767,78379937,W,21367,Michancy,agreed! the russian connection to this administration must be fully investigated. flynn is the ...,Jacky Rosen,Congress_Democratic
2,1682768,78379937,W,21367,Andrea,please push chaffez and cummings to look into this!,Jacky Rosen,Congress_Democratic
3,1682769,78379937,W,21367,Tina,i could not agree more!,Jacky Rosen,Congress_Democratic
4,1682770,78379937,W,21368,Cathy,"i imagine that is followed by ""when qualifited.""",Jacky Rosen,Congress_Democratic


In [0]:
all_responses = pd.concat([train_df,test_df,dev_df])

In [0]:
all_unique_posters_train = train_df['op_id'].drop_duplicates().values
all_unique_posters_dev = dev_df['op_id'].drop_duplicates().values
all_unique_posters_test = test_df['op_id'].drop_duplicates().values
all_unique_posters_train

In [22]:
all_unique_posters = np.concatenate((all_unique_posters_train,all_unique_posters_dev,all_unique_posters_test))
len(all_unique_posters)

402

In [25]:
# shuffle with random seed 
all_unique_posters2 = copy.deepcopy(all_unique_posters)
random.Random(7).shuffle(all_unique_posters2)
all_unique_posters2[:5]

array([92713922, 38408437,  3812062,  3613205, 49930535])

In [0]:
# Create indices for 70-15-15 split.
seventy_percent_split = math.floor(.7 * len(all_unique_posters))
fifteen_percent_split = seventy_percent_split + math.floor(.15*len(all_unique_posters))
train_idxs = all_unique_posters2[:seventy_percent_split]
test_idxs = all_unique_posters2[seventy_percent_split:fifteen_percent_split]
dev_idxs = all_unique_posters2[fifteen_percent_split:]

In [0]:
def determine_gender_split_of_data_subsets(idx_list, list_title):
    posters = pd.DataFrame({'op_id': idx_list})
    posters = posters.merge(all_responses[['op_id','op_gender']], on='op_id')
    posters = posters.drop_duplicates(subset='op_id')
    percent_male = len(posters[posters.op_gender=='M']) / len(posters)
    print("Proportion male in {} set: {:.3f}".format(list_title, percent_male))

In [33]:
determine_gender_split_of_data_subsets(train_idxs, "Train")
determine_gender_split_of_data_subsets(test_idxs, "Test")
determine_gender_split_of_data_subsets(dev_idxs, "Dev")

Proportion male in Train set: 0.747
Proportion male in Test set: 0.800
Proportion male in Dev set: 0.787


In [0]:
train_rows = all_responses[all_responses.op_id.isin(train_idxs)]
test_rows = all_responses[all_responses.op_id.isin(test_idxs)]
dev_rows = all_responses[all_responses.op_id.isin(dev_idxs)]

In [36]:
def determine_relative_dataset_sizes(train_rows, test_rows, dev_rows):
    train_size = len(train_rows)
    test_size = len(test_rows)
    dev_size = len(dev_rows)
    full_size = train_size + dev_size + test_size
    print("Proportion train: {:.3f}".format(train_size / full_size))
    print("Proportion test: {:.3f}".format(test_size / full_size))
    print("Proportion dev: {:.3f}".format(dev_size / full_size))
    
determine_relative_dataset_sizes(train_rows, test_rows, dev_rows)

Proportion train: 0.767
Proportion test: 0.087
Proportion dev: 0.145


In [0]:
train_df = train_rows
dev_df = dev_rows

In [39]:
# Remove 1 and 2 word responses from train set.
# Split response string by spaces.
train_df['split_response'] = train_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
mask = (train_df['split_response'].str.len() > 2) 
train_df = train_df.loc[mask]
len(train_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


9348779

In [40]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
mask2 = (dev_df['split_response'].str.len() > 2) 
dev_df = dev_df.loc[mask2]
len(dev_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1815559

In [41]:
# Limit the number of responses per post to 50.
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3831551, 9)

In [75]:
import gc
gc.collect()

41

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [46]:
train_df.op_gender.value_counts()

M    2755275
W    1076276
Name: op_gender, dtype: int64

In [47]:
dev_df.op_gender.value_counts()

M    1494771
W     320788
Name: op_gender, dtype: int64

In [0]:
###########################
# Execute this if want to do oversampling.
###########################
# from sklearn.utils import resample

# male = train_df[train_df.op_gender=='M']
# female = train_df[train_df.op_gender=='W']

# female_upsampled = resample(female,
#                           replace=True, # sample with replacement
#                           n_samples=len(male), # match number in majority class
#                           random_state=27) # reproducible results
# upsampled_train = pd.concat([male,female_upsampled])
# print(upsampled_train.op_gender.value_counts())


In [0]:
# replace `train_df` with `upsampled_train` if using oversampling.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

# replace `train_df` with `upsampled_train` if using upsampling.
new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [52]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 05 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [55]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]    5.1 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [58]:
embedding_matrix.shape

(578459, 100)

In [59]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           57845900  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [63]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3831551 samples, validate on 1815559 samples
Epoch 1/30
3831551/3831551 [==============================] - 102s 27us/sample - loss: 0.5294 - acc: 0.7522 - val_loss: 0.4512 - val_acc: 0.8240
Epoch 2/30
3831551/3831551 [==============================] - 103s 27us/sample - loss: 0.5132 - acc: 0.7598 - val_loss: 0.4541 - val_acc: 0.8256
Epoch 3/30
3831551/3831551 [==============================] - 104s 27us/sample - loss: 0.5077 - acc: 0.7625 - val_loss: 0.4500 - val_acc: 0.8218
Epoch 4/30
3831551/3831551 [==============================] - 106s 28us/sample - loss: 0.5044 - acc: 0.7640 - val_loss: 0.4524 - val_acc: 0.8248
Epoch 5/30
3831551/3831551 [==============================] - 112s 29us/sample - loss: 0.5019 - acc: 0.7651 - val_loss: 0.4503 - val_acc: 0.8217
Epoch 6/30
3831551/3831551 [==============================] - 105s 27us/sample - loss: 0.5001 - acc: 0.7659 - val_loss: 0.4463 - val_acc: 0.8259
Epoch 7/30
3831551/3831551 [==============================] - 105s 27us/samp

In [64]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7735
Testing Accuracy:  0.8185


In [65]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6682940344838586


In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [70]:
wrong_preds.op_gender.value_counts()

W    273736
M     55808
Name: op_gender, dtype: int64

In [71]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.0566547272768332

In [72]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
1956642,11006034,86569077,W,389607,Janet,"i plan to watch it on demand tonight. john is awesome. and senator gilibrand, if you like and tr...",Kirsten Gillibrand,Congress_Democratic,"[i, plan, to, watch, it, on, demand, tonight., john, is, awesome., and, senator, gilibrand,, if,...",0.888917,M
4689002,6270599,17141989,W,229988,Dominique,kayleigh louise pitts,Maxine Waters,Congress_Democratic,"[kayleigh, louise, pitts]",0.740280,M
3235012,4013777,53750474,W,135955,Diana,you are doing a great job!!!,Carol Shea-Porter,Congress_Democratic,"[you, are, doing, a, great, job!!!]",0.651075,M
2031661,2104837,56883970,W,35038,Jim,"tim, you better start caring, and soon....",Diane Black,Congress_Republican,"[tim,, you, better, start, caring,, and, soon....]",0.917917,M
1995656,11045048,86569077,W,390009,Jon,i like the warrent plan better. and what makes anyone think that conservative colleges have any ...,Kirsten Gillibrand,Congress_Democratic,"[i, like, the, warrent, plan, better., and, what, makes, anyone, think, that, conservative, coll...",0.628119,M
7633338,10672726,94031602,M,384028,Jim,"""she does not have a track record of leadership or trustworthiness. she has not the woman for th...",Trey Gowdy,Congress_Republican,"[""she, does, not, have, a, track, record, of, leadership, or, trustworthiness., she, has, not, t...",0.441523,W
2009399,11058791,86569077,W,390113,Maila,wonderful picture. I am dave connors grandmothe- in- law so follow your progress. if i lived in ...,Kirsten Gillibrand,Congress_Democratic,"[wonderful, picture., I, am, dave, connors, grandmothe-, in-, law, so, follow, your, progress., ...",0.656824,M
2004498,11053890,86569077,W,390078,Ann,would passing the era help us all?,Kirsten Gillibrand,Congress_Democratic,"[would, passing, the, era, help, us, all?]",0.622804,M
7349015,10388403,94031602,M,383858,Tammy,"thank you, happy thanksgiving!",Trey Gowdy,Congress_Republican,"[thank, you,, happy, thanksgiving!]",0.484340,W
8328681,11811872,8332544,M,430573,Margie,where is the governor of california?,Doug LaMalfa,Congress_Republican,"[where, is, the, governor, of, california?]",0.392192,W


In [73]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
1951491,11000883,86569077,W,389558,Sergio,thank you for doing so over mr. schumer. you are a real patriot!,Kirsten Gillibrand,Congress_Democratic,"[thank, you, for, doing, so, over, mr., schumer., you, are, a, real, patriot!]",0.999968,M
4701022,6282619,17141989,W,230167,Gloria,love the work you & congressman pascrell do & do carry on>>smiles,Maxine Waters,Congress_Democratic,"[love, the, work, you, &, congressman, pascrell, do, &, do, carry, on>>smiles]",0.999904,M
4321879,5824733,10588553,W,193358,Lee,rand is wayyy off base...but let him split the repukes! go rand go lol,Claire McCaskill,Congress_Democratic,"[rand, is, wayyy, off, base...but, let, him, split, the, repukes!, go, rand, go, lol]",0.999897,M
4679751,6261348,17141989,W,229967,Rose,please urge your republican senators to put national security ahead of party politics and take t...,Maxine Waters,Congress_Democratic,"[please, urge, your, republican, senators, to, put, national, security, ahead, of, party, politi...",0.999896,M
2091275,2351736,82045172,W,52334,Cisco,"good luck to you, issa, chaffetz, and polis today!",Zoe Lofgren,Congress_Democratic,"[good, luck, to, you,, issa,, chaffetz,, and, polis, today!]",0.999875,M
1951070,11000462,86569077,W,389558,Lynn,thanks for being braver than sen. schumer,Kirsten Gillibrand,Congress_Democratic,"[thanks, for, being, braver, than, sen., schumer]",0.999829,M
2025835,11075227,86569077,W,390288,Bonnie,re mtp: even though michelle bachmann was rude and off-point you presented a calm and reasoned ...,Kirsten Gillibrand,Congress_Democratic,"[re, mtp:, even, though, michelle, bachmann, was, rude, and, off-point, you, presented, a, calm,...",0.999788,M
2860133,3633492,3080550,W,123606,Tiny,from miap & patriot guard riders thank you for your time and effort & support. lindsey helping...,Martha McSally,Congress_Republican,"[from, miap, &, patriot, guard, riders, thank, you, for, your, time, and, effort, &, support., l...",0.999783,M
4342170,5845024,10588553,W,193450,Carl,"this happened before and yes, there are people who actually believe this crap - more than you th...",Claire McCaskill,Congress_Democratic,"[this, happened, before, and, yes,, there, are, people, who, actually, believe, this, crap, -, m...",0.999779,M
2037406,2110582,56883970,W,35314,Howard,thank you for standing with senator paul. i wish we had at least one tennessee senator who would...,Diane Black,Congress_Republican,"[thank, you, for, standing, with, senator, paul., i, wish, we, had, at, least, one, tennessee, s...",0.999764,M


In [74]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response,probs,preds
5588873,7404540,25380823,M,280541,Nadine,"""the first female campaign manager of either major party to win a presidential general election ...",Dave Brat,Congress_Republican,"[""the, first, female, campaign, manager, of, either, major, party, to, win, a, presidential, gen...",0.000024,W
3295306,4095002,59295258,M,144748,Brenda,could you run again marsha blackheart?,Steve Cohen,Congress_Democratic,"[could, you, run, again, marsha, blackheart?]",0.000048,W
7412742,10452130,94031602,M,383895,Carrie,"thank you constance,,,,america needs more people like you",Trey Gowdy,Congress_Republican,"[thank, you, constance,,,,america, needs, more, people, like, you]",0.000082,W
4396006,5921673,91743222,M,200953,Pastores,rosa harris wounded worrier program dc 703-554-4697. she is the coordinator for the wwp walter ...,Brian Mast,Congress_Republican,"[rosa, harris, wounded, worrier, program, dc, 703-554-4697., she, is, the, coordinator, for, the...",0.000196,W
9708052,13636966,67024958,M,530350,Sharon,you have us in your corner and ready to work with you in any way we can. together we are stron...,Richard Blumenthal,Congress_Democratic,"[you, have, us, in, your, corner, and, ready, to, work, with, you, in, any, way, we, can., toget...",0.000256,W
9707280,13636194,67024958,M,530340,Sarah,"dear senator blumenthal, many of our brace young men and women who have served this nation are...",Richard Blumenthal,Congress_Democratic,"[dear, senator, blumenthal,, many, of, our, brace, young, men, and, women, who, have, served, th...",0.000269,W
7835122,10939333,25181563,M,387009,Suzanne,"happy birthday, steny! hope this new year in your life is as special as you are! huge birthday ...",Steny Hoyer,Congress_Democratic,"[happy, birthday,, steny!, hope, this, new, year, in, your, life, is, as, special, as, you, are!...",0.000284,W
9717118,13646032,67024958,M,530611,Kathy,stop the nuke deal...it has sick that they are going to police themselves. do something other th...,Richard Blumenthal,Congress_Democratic,"[stop, the, nuke, deal...it, has, sick, that, they, are, going, to, police, themselves., do, som...",0.000289,W
5673941,7642761,12336661,M,295066,Justin,"i do but i live in spokane, wa. bummer, well wishes cap't higgins! hope you win and make it to dc!",Clay Higgins,Congress_Republican,"[i, do, but, i, live, in, spokane,, wa., bummer,, well, wishes, cap't, higgins!, hope, you, win,...",0.000301,W
7364884,10404272,94031602,M,383866,Bob,"if you have had a positive impact on just one life in this world, then you have done it right in...",Trey Gowdy,Congress_Republican,"[if, you, have, had, a, positive, impact, on, just, one, life, in, this, world,, then, you, have...",0.000331,W
